In [158]:
from SPARQLWrapper import SPARQLWrapper, XML, JSON
import pandas as pd 
import bs4 as bs
import re
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

In [2]:
#hard code the source for the example
source = "http://sparql.hegroup.org/sparql/"

To customize, change the rdfs:subclass of <http://edamontology.org/topic_0003> line to an IRI of a parent class of your choosing.

In [179]:
#Function to find all missing wikis (JSON Output)
def missing_wikis(source):
    sparql = SPARQLWrapper(source)
    sparql.setReturnFormat(JSON)
    sparql.setQuery("""
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>
    PREFIX edam:<http://edamontology.org/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT DISTINCT ?entity ?label ?seealso WHERE {
        ?entity rdfs:subClassOf+ <http://edamontology.org/topic_0003> .
        ?entity rdfs:label ?label .

        FILTER NOT EXISTS {
            ?entity rdfs:seeAlso ?seealso .
            FILTER (regex(str(?seealso), "wikipedia", "i")) .
        }
    }
    """)
    try: 
        ret = sparql.queryAndConvert()
        results = []
        for r in ret["results"]["bindings"]:
            results.append(r)
        entity = []
        label = []
        for result in results: 
            entity.append(result["entity"]['value'])
            label.append(result["label"]["value"])

        df = pd.DataFrame({"entity": entity, "label": label})
        pd.set_option('display.max_rows', None)

        return df
    except: 
        print("Oh Noes!")

In [180]:
missing_wikis(source)

,entity,label
0,http://edamontology.org/topic_3966,Vaccinology
1,http://edamontology.org/topic_3967,Immunomics
2,http://edamontology.org/topic_3974,Epistasis
3,http://edamontology.org/topic_3837,Metagenomic sequencing
4,http://edamontology.org/topic_0123,Protein properties
5,http://edamontology.org/topic_0157,"Sequence composition, complexity and repeats"
6,http://edamontology.org/topic_0160,"Sequence sites, features and motifs"
7,http://edamontology.org/topic_0166,Protein structural motifs and surfaces
8,http://edamontology.org/topic_0219,"Data submission, annotation and curation"
9,http://edamontology.org/topic_0632,Probes and primers


Below is an older version of the code that converted an xml output to a dataframe. 

In [160]:
def missing_wiki_info(source):
    sparql = SPARQLWrapper(source)
    sparql.setReturnFormat(XML)
    sparql.setQuery("""
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>
    PREFIX edam:<http://edamontology.org/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT DISTINCT ?entity ?label ?seealso WHERE {
        ?entity rdfs:subClassOf+ <http://edamontology.org/topic_0003> .
        ?entity rdfs:label ?label .

        FILTER NOT EXISTS {
            ?entity rdfs:seeAlso ?seealso .
            FILTER (regex(str(?seealso), "wikipedia", "i")) .
        }
    }
    """)
    results = sparql.query().convert()
    results = results.toxml()
    
    file = bs.BeautifulSoup(results, "lxml")
    find_table = file.find_all('binding')
    labels = []
    uris = []
    for entry in find_table: 
        labels.append(entry.find_all('literal'))
        uris.append(entry.find_all('uri'))
        
    for label in labels: 
        if label == []: 
            labels.remove(label)
    for uri in uris: 
        if uri == []:
            uris.remove(uri)
            
    flat_labels = [item for sublist in labels for item in sublist]
    flat_labels = list(map(str, flat_labels))
    labels_cleaned = []
    for i in range(len(flat_labels)):
        line = striphtml(flat_labels[i])
        labels_cleaned.append(line)
    flat_uris = [item for sublist in uris for item in sublist]
    flat_uris = list(map(str, flat_uris))
    
    uris_cleaned = []
    for i in range(len(flat_uris)):
        line = striphtml(flat_uris[i])
        uris_cleaned.append(line)
    df = pd.DataFrame({"entity": uris_cleaned, "label": labels_cleaned})
    return df 

In [163]:
missing_wiki_info(source)

,entity,label
0,http://edamontology.org/topic_3966,Vaccinology
1,http://edamontology.org/topic_3967,Immunomics
2,http://edamontology.org/topic_3974,Epistasis
3,http://edamontology.org/topic_3837,Metagenomic sequencing
4,http://edamontology.org/topic_0123,Protein properties
...,...,...
70,http://edamontology.org/operation_3923,Genome resequencing
71,http://edamontology.org/topic_3892,Biomolecular simulation
72,http://edamontology.org/topic_3954,Echography
73,http://edamontology.org/topic_3955,Fluxomics
